# Starbucks Capstone Challenge

### Introduction

This data set contains simulated data that mimics customer behavior on the Starbucks rewards mobile app. Once every few days, Starbucks sends out an offer to users of the mobile app. An offer can be merely an advertisement for a drink or an actual offer such as a discount or BOGO (buy one get one free). Some users might not receive any offer during certain weeks. 

Not all users receive the same offer, and that is the challenge to solve with this data set.

Your task is to combine transaction, demographic and offer data to determine which demographic groups respond best to which offer type. This data set is a simplified version of the real Starbucks app because the underlying simulator only has one product whereas Starbucks actually sells dozens of products.

Every offer has a validity period before the offer expires. As an example, a BOGO offer might be valid for only 5 days. You'll see in the data set that informational offers have a validity period even though these ads are merely providing information about a product; for example, if an informational offer has 7 days of validity, you can assume the customer is feeling the influence of the offer for 7 days after receiving the advertisement.

You'll be given transactional data showing user purchases made on the app including the timestamp of purchase and the amount of money spent on a purchase. This transactional data also has a record for each offer that a user receives as well as a record for when a user actually views the offer. There are also records for when a user completes an offer. 

Keep in mind as well that someone using the app might make a purchase through the app without having received an offer or seen an offer.

### Example

To give an example, a user could receive a discount offer buy 10 dollars get 2 off on Monday. The offer is valid for 10 days from receipt. If the customer accumulates at least 10 dollars in purchases during the validity period, the customer completes the offer.

However, there are a few things to watch out for in this data set. Customers do not opt into the offers that they receive; in other words, a user can receive an offer, never actually view the offer, and still complete the offer. For example, a user might receive the "buy 10 dollars get 2 dollars off offer", but the user never opens the offer during the 10 day validity period. The customer spends 15 dollars during those ten days. There will be an offer completion record in the data set; however, the customer was not influenced by the offer because the customer never viewed the offer.

### Cleaning

This makes data cleaning especially important and tricky.

You'll also want to take into account that some demographic groups will make purchases even if they don't receive an offer. From a business perspective, if a customer is going to make a 10 dollar purchase without an offer anyway, you wouldn't want to send a buy 10 dollars get 2 dollars off offer. You'll want to try to assess what a certain demographic group will buy when not receiving any offers.

### Final Advice

Because this is a capstone project, you are free to analyze the data any way you see fit. For example, you could build a machine learning model that predicts how much someone will spend based on demographics and offer type. Or you could build a model that predicts whether or not someone will respond to an offer. Or, you don't need to build a machine learning model at all. You could develop a set of heuristics that determine what offer you should send to each customer (i.e., 75 percent of women customers who were 35 years old responded to offer A vs 40 percent from the same demographic to offer B, so send offer A).

# Data Sets

The data is contained in three files:

* portfolio.json - containing offer ids and meta data about each offer (duration, type, etc.)
* profile.json - demographic data for each customer
* transcript.json - records for transactions, offers received, offers viewed, and offers completed

Here is the schema and explanation of each variable in the files:

**portfolio.json**
* id (string) - offer id
* offer_type (string) - type of offer ie BOGO, discount, informational
* difficulty (int) - minimum required spend to complete an offer
* reward (int) - reward given for completing an offer
* duration (int) - time for offer to be open, in days
* channels (list of strings)

**profile.json**
* age (int) - age of the customer 
* became_member_on (int) - date when customer created an app account
* gender (str) - gender of the customer (note some entries contain 'O' for other rather than M or F)
* id (str) - customer id
* income (float) - customer's income

**transcript.json**
* event (str) - record description (ie transaction, offer received, offer viewed, etc.)
* person (str) - customer id
* time (int) - time in hours since start of test. The data begins at time t=0
* value - (dict of strings) - either an offer id or transaction amount depending on the record

**Note:** If you are using the workspace, you will need to go to the terminal and run the command `conda update pandas` before reading in the files. This is because the version of pandas in the workspace cannot read in the transcript.json file correctly, but the newest version of pandas can. You can access the termnal from the orange icon in the top left of this notebook.  

You can see how to access the terminal and how the install works using the two images below.  First you need to access the terminal:

<img src="pic1.png"/>

Then you will want to run the above command:

<img src="pic2.png"/>

Finally, when you enter back into the notebook (use the jupyter icon again), you should be able to run the below cell without any errors.

In [88]:
import pandas as pd
import numpy as np
import math
import json
%matplotlib inline

# read in the json files
portfolio = pd.read_json('data/portfolio.json', orient='records', lines=True)
profile = pd.read_json('data/profile.json', orient='records', lines=True)
transcript = pd.read_json('data/transcript.json', orient='records', lines=True)

In [89]:
transcript.describe()

,time
count,306534.000000
mean,366.382940
std,200.326314
min,0.000000
25%,186.000000
50%,408.000000
75%,528.000000
max,714.000000


In [90]:
transcript.columns

Index(['person', 'event', 'value', 'time'], dtype='object')

Lets figure out the distribution of the data. 
It will be helpful to find out the proportion of data that is short vs long-term

In [91]:
portfolio.describe()

,reward,difficulty,duration
count,10.000000,10.000000,10.000000
mean,4.200000,7.700000,6.500000
std,3.583915,5.831905,2.321398
min,0.000000,0.000000,3.000000
25%,2.000000,5.000000,5.000000
50%,4.000000,8.500000,7.000000
75%,5.000000,10.000000,7.000000
max,10.000000,20.000000,10.000000


In [92]:
portfolio.head()

,reward,channels,difficulty,duration,offer_type,id
0,10,"[email, mobile, social]",10,7,bogo,ae264e3637204a6fb9bb56bc8210ddfd
1,10,"[web, email, mobile, social]",10,5,bogo,4d5c57ea9a6940dd891ad53e9dbe8da0
2,0,"[web, email, mobile]",0,4,informational,3f207df678b143eea3cee63160fa8bed
3,5,"[web, email, mobile]",5,7,bogo,9b98b8c7a33c4b65b9aebfe6a799e6d9
4,5,"[web, email]",20,10,discount,0b1e1539f2cc45b7b9fa7c272da2e1d7


In [93]:
portfolio.shape

(10, 6)

In [94]:
transcript.shape

(306534, 4)

In [95]:
transcript.head()

,person,event,value,time
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},0
1,a03223e636434f42ac4c3df47e8bac43,offer received,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'},0
2,e2127556f4f64592b11af22de27a7932,offer received,{'offer id': '2906b810c7d4411798c6938adc9daaa5'},0
3,8ec6ce2a7e7949b1bf142def7d0e0586,offer received,{'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'},0
4,68617ca6246f4fbc85e91a2a49552598,offer received,{'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'},0


In [96]:
transcript['event'][0]

'offer received'

In [97]:
portfolio_subset = portfolio[['id', 'offer_type', 'difficulty']]

In [98]:
portfolio_subset

,id,offer_type,difficulty
0,ae264e3637204a6fb9bb56bc8210ddfd,bogo,10
1,4d5c57ea9a6940dd891ad53e9dbe8da0,bogo,10
2,3f207df678b143eea3cee63160fa8bed,informational,0
3,9b98b8c7a33c4b65b9aebfe6a799e6d9,bogo,5
4,0b1e1539f2cc45b7b9fa7c272da2e1d7,discount,20
5,2298d6c36e964ae4a3e7e9706d1fb8c2,discount,7
6,fafdcd668e3743c1bb461111dcafc2a4,discount,10
7,5a8bc65990b245e5a138643cd4eb9837,informational,0
8,f19421c1d4aa40978ebb69ca19b0e20d,bogo,5
9,2906b810c7d4411798c6938adc9daaa5,discount,10


In [99]:
transcript_subset = transcript.loc[:, ['value', 'event']]

In [100]:
transcript_subset['value']

0         {'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'}
1         {'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'}
2         {'offer id': '2906b810c7d4411798c6938adc9daaa5'}
3         {'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'}
4         {'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'}
                                ...                       
306529                      {'amount': 1.5899999999999999}
306530                                    {'amount': 9.53}
306531                                    {'amount': 3.61}
306532                      {'amount': 3.5300000000000002}
306533                                    {'amount': 4.05}
Name: value, Length: 306534, dtype: object

In [101]:
transcript_subset = transcript_subset.loc[transcript_subset['value'].apply(lambda x: 'offer id' in x)]

In [102]:
transcript_subset['id'] = transcript_subset['value'].apply(lambda x: list(x.values())[0])

In [103]:
transcript_subset.head()

,value,event,id
0,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},offer received,9b98b8c7a33c4b65b9aebfe6a799e6d9
1,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'},offer received,0b1e1539f2cc45b7b9fa7c272da2e1d7
2,{'offer id': '2906b810c7d4411798c6938adc9daaa5'},offer received,2906b810c7d4411798c6938adc9daaa5
3,{'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'},offer received,fafdcd668e3743c1bb461111dcafc2a4
4,{'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'},offer received,4d5c57ea9a6940dd891ad53e9dbe8da0


In [104]:
portfolio_subset.head()

,id,offer_type,difficulty
0,ae264e3637204a6fb9bb56bc8210ddfd,bogo,10
1,4d5c57ea9a6940dd891ad53e9dbe8da0,bogo,10
2,3f207df678b143eea3cee63160fa8bed,informational,0
3,9b98b8c7a33c4b65b9aebfe6a799e6d9,bogo,5
4,0b1e1539f2cc45b7b9fa7c272da2e1d7,discount,20


In [105]:
pd.merge(transcript_subset, portfolio_subset, on='id', how='inner')

,value,event,id,offer_type,difficulty
0,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},offer received,9b98b8c7a33c4b65b9aebfe6a799e6d9,bogo,5
1,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'},offer received,0b1e1539f2cc45b7b9fa7c272da2e1d7,discount,20
2,{'offer id': '2906b810c7d4411798c6938adc9daaa5'},offer received,2906b810c7d4411798c6938adc9daaa5,discount,10
3,{'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'},offer received,fafdcd668e3743c1bb461111dcafc2a4,discount,10
4,{'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'},offer received,4d5c57ea9a6940dd891ad53e9dbe8da0,bogo,10
...,...,...,...,...,...
133997,{'offer id': '5a8bc65990b245e5a138643cd4eb9837'},offer viewed,5a8bc65990b245e5a138643cd4eb9837,informational,0
133998,{'offer id': '3f207df678b143eea3cee63160fa8bed'},offer viewed,3f207df678b143eea3cee63160fa8bed,informational,0
133999,{'offer id': '5a8bc65990b245e5a138643cd4eb9837'},offer viewed,5a8bc65990b245e5a138643cd4eb9837,informational,0
134000,{'offer id': '3f207df678b143eea3cee63160fa8bed'},offer viewed,3f207df678b143eea3cee63160fa8bed,informational,0


In [106]:
pd.merge?

Signature:
pd.merge(
    left: 'DataFrame | Series',
    right: 'DataFrame | Series',
    how: 'MergeHow' = 'inner',
    on: 'IndexLabel | AnyArrayLike | None' = None,
    left_on: 'IndexLabel | AnyArrayLike | None' = None,
    right_on: 'IndexLabel | AnyArrayLike | None' = None,
    left_index: 'bool' = False,
    right_index: 'bool' = False,
    sort: 'bool' = False,
    suffixes: 'Suffixes' = ('_x', '_y'),
    copy: 'bool | None' = None,
    indicator: 'str | bool' = False,
    validate: 'str | None' = None,
) -> 'DataFrame'
Docstring:
Merge DataFrame or named Series objects with a database-style join.

A named Series object is treated as a DataFrame with a single named column.

The join is done on columns or indexes. If joining columns on
columns, the DataFrame indexes *will be ignored*. Otherwise if joining indexes
on indexes or indexes on a column or columns, the index will be passed on.
When performing a cross merge, no column specifications to merge on are
allowed.

.. warning: